In [22]:
#from sklearn.preprocessing import LabelEncoder
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder

# Sem WHOQOL:
base1 = pd.read_csv('Datasets/dataframe_socioeconomico_Niveis.csv')
data1 = base1.drop('Nivel_MHI', axis=1)
target1 = base1['Nivel_MHI']
le = LabelEncoder()
target1 = le.fit_transform(target1)

# Realizar Random Under-Sampling
sampler = RandomUnderSampler()
data1_re, target1_re = sampler.fit_resample(data1, target1)


# com todas as respostas do WHOQOL
base2 = pd.read_csv('Datasets/df_social_whoqol_tratado_1.csv')
data2 = base2.drop('Nivel_MHI', axis=1)
target2 = base2['Nivel_MHI']
target2 = le.fit_transform(target2)
sampler = RandomUnderSampler()
data2_re, target2_re = sampler.fit_resample(data2, target2)


# só com o indice dos dominiosdo WHOQOL, concatenar base1 com final da base2
data3 = pd.concat([data1, base2.loc[:,['CR','CS','CT','CU']]], axis=1)
target3 = base2['Nivel_MHI']
target3 = le.fit_transform(target3)
sampler = RandomUnderSampler()
data3_re, target3_re = sampler.fit_resample(data3, target3)


# Só com a qualidade de vida do WHOQOL
data4 = pd.concat([data1, base2['BR']], axis=1)
target4 = base2['Nivel_MHI']
target4 = le.fit_transform(target4)
sampler = RandomUnderSampler()
data4_re, target4_re = sampler.fit_resample(data4, target4)

# Qualidade de Vida Geral ( Media das duas primeiras questões do WHOQOL)
# Pegar a média das duas primeiras colunas da base2, pelo indice
data5 = pd.concat([data1, base2.loc[:,['BR','BS']].mean(axis=1)], axis=1)
target5 = base2['Nivel_MHI']
target5 = le.fit_transform(target5)
sampler = RandomUnderSampler()
data5.columns = data5.columns.astype(str)
data5_re, target5_re = sampler.fit_resample(data5, target5)

# Media de todos os dominios do WHOQOL
data6 = pd.concat([data1, base2.loc[:,['CR','CS','CT','CU']].mean(axis=1)], axis=1)
target6 = base2['Nivel_MHI']
target6 = le.fit_transform(target6)
sampler = RandomUnderSampler()
data6.columns = data6.columns.astype(str)
data6_re, target6_re = sampler.fit_resample(data6, target6)

In [23]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate 

# Função para calcular a entropia de um conjunto de dados
def entropy(data):
    class_labels, counts = np.unique(data, return_counts=True)
    total_instances = len(data)
    probabilities = counts / total_instances
    entropy_val = -np.sum(probabilities * np.log2(probabilities))

    return entropy_val

# Função para calcular o ganho de informação de um atributo
def information_gain(data, attribute, target):
    attribute_values = data[attribute].unique()
    total_instances = len(data)
    info_gain = entropy(data[target])

    for value in attribute_values:
        subset = data[data[attribute] == value]
        prob = len(subset) / total_instances
        info_gain -= prob * entropy(subset[target])
    return info_gain

# Função para calcular os ganhos de informação de todos os atributos em relação ao atributo alvo
def calculate_information_gains(data, target):
    all_attributes = data.columns
    gains = {attribute: information_gain(data, attribute, target) for attribute in all_attributes}
    result_df = pd.DataFrame(gains.items(), columns=['Atributo', 'Ganho de Informação'])
    result_df = result_df.sort_values(by='Ganho de Informação', ascending=False)
    return result_df

# Ler o dataframe e pré-processar o atributo alvo
base1 = pd.read_csv('Datasets/dataframe_socioeconomico_Niveis.csv')
data1 = base1.drop('Nivel_MHI', axis=1)
target1 = base1['Nivel_MHI']
le = LabelEncoder()
target1 = le.fit_transform(target1)

# Converter o atributo alvo de volta para rótulos de classes
target_labels = le.inverse_transform(target1)
data1['Nivel_MHI'] = target_labels

# Calcular os ganhos de informação
#esult_df = calculate_information_gains(data1, 'Nivel_MHI')
result_df = calculate_information_gains(data1, 'Nivel_MHI')
result_df

,Atributo,Ganho de Informação
54,Nivel_MHI,1.268220
26,BP,0.055492
1,C,0.039201
23,AZ,0.036363
7,O,0.022827
13,AO,0.021020
5,M,0.020295
14,AP,0.017528
44,J_5.0,0.016862
19,AV,0.015995


In [24]:
import pandas as pd
import numpy as np
import math
import time
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Função para calcular a entropia de um conjunto de dados
def entropy(data):
    class_labels, counts = np.unique(data, return_counts=True)
    total_instances = len(data)
    probabilities = counts / total_instances
    entropy_val = -np.sum(probabilities * np.log2(probabilities))

    return entropy_val

# Função para calcular o ganho de informação de um atributo
def information_gain(data, attribute, target):
    attribute_values = data[attribute].unique()
    total_instances = len(data)
    info_gain = entropy(data[target])

    for value in attribute_values:
        subset = data[data[attribute] == value]
        prob = len(subset) / total_instances
        info_gain -= prob * entropy(subset[target])

    return info_gain

# Função para calcular os ganhos de informação de todos os atributos em relação ao atributo alvo
def calculate_information_gains(data, target):
    all_attributes = data.columns.drop(target)
    gains = {attribute: information_gain(data, attribute, target) for attribute in all_attributes}
    result_df = pd.DataFrame(gains.items(), columns=['Atributo', 'Ganho de Informação'])
    result_df = result_df.sort_values(by='Ganho de Informação', ascending=False)
    return result_df

# Função para normalizar os valores dos ganhos de informação
def normalize_gains(result_df):
    scaler = MinMaxScaler()
    result_df['Ganho de Informação Normalizado'] = scaler.fit_transform(result_df[['Ganho de Informação']])
    return result_df

# Ler o dataframe e pré-processar o atributo alvo
base1 = pd.read_csv('Datasets/dataframe_socioeconomico_Niveis.csv')
data1 = base1.drop('Nivel_MHI', axis=1)
target1 = base1['Nivel_MHI']
le = LabelEncoder()
target1 = le.fit_transform(target1)

# Converter o atributo alvo de volta para rótulos de classes
target_labels = le.inverse_transform(target1)
data1['Nivel_MHI'] = target_labels


# Calcular os ganhos de informação
result_df = calculate_information_gains(data1, 'Nivel_MHI')

# Normalizar os ganhos de informação
result_df = normalize_gains(result_df)

# Imprimir o resultado
print(result_df)

   Atributo  Ganho de Informação  Ganho de Informação Normalizado
26       BP             0.055492                         1.000000
1         C             0.039201                         0.706079
23       AZ             0.036363                         0.654880
7         O             0.022827                         0.410669
13       AO             0.021020                         0.378063
5         M             0.020295                         0.364981
14       AP             0.017528                         0.315055
44    J_5.0             0.016862                         0.303043
19       AV             0.015995                         0.287401
25       BO             0.015834                         0.284501
8         U             0.014742                         0.264791
24       BD             0.014318                         0.257149
3         H             0.012816                         0.230046
4         L             0.012538                         0.225031
16       A

In [27]:
import pandas as pd
import numpy as np
import math
import time
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Função para calcular a entropia de um conjunto de dados
def entropy(data):
    class_labels, counts = np.unique(data, return_counts=True)
    total_instances = len(data)
    probabilities = counts / total_instances
    entropy_val = -np.sum(probabilities * np.log2(probabilities))

    return entropy_val

# Função para calcular o ganho de informação de um atributo
def information_gain(data, attribute, target):
    attribute_values = data[attribute].unique()
    total_instances = len(data)
    info_gain = entropy(data[target])

    for value in attribute_values:
        subset = data[data[attribute] == value]
        prob = len(subset) / total_instances
        info_gain -= prob * entropy(subset[target])

    return info_gain

# Função para calcular os ganhos de informação de todos os atributos em relação ao atributo alvo
def calculate_information_gains(data, target):
    all_attributes = data.columns.drop(target)
    gains = {attribute: information_gain(data, attribute, target) for attribute in all_attributes}
    result_df = pd.DataFrame(gains.items(), columns=['Atributo', 'Ganho de Informação'])
    result_df = result_df.sort_values(by='Ganho de Informação', ascending=False)
    return result_df

# Função para normalizar os valores dos ganhos de informação
def normalize_gains(result_df):
    scaler = MinMaxScaler()
    result_df['Ganho de Informação Normalizado'] = scaler.fit_transform(result_df[['Ganho de Informação']])
    return result_df

# Função para ler o arquivo "titulos_das_colunas.txt" e criar um dicionário com o mapeamento dos índices para os nomes das colunas
def read_column_titles(file_path):
    column_titles = {}
    questionarios = {}
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            if lines[i].startswith("Título da Coluna:"):
                column_title = lines[i].split(":")[1].strip()
                coordinate = lines[i+1].split(":")[1].strip()
                questionario = lines[i+2].split(":")[1].strip()
                column_titles[coordinate] = column_title
                questionarios[coordinate] = questionario
                
                i += 2
            else:
                i += 1
    return column_titles, questionarios

# Ler o arquivo "titulos_das_colunas.txt" e criar o dicionário de mapeamento dos índices para os nomes das colunas
column_titles,questionarios = read_column_titles('titulos_das_colunas.txt')

# com todas as respostas do WHOQOL
base2 = pd.read_csv('Datasets/df_social_whoqol_tratado_1.csv')
data2 = base2.drop('Nivel_MHI', axis=1)
target2 = base2['Nivel_MHI']
target2 = le.fit_transform(target2)

# Converter o atributo alvo de volta para rótulos de classes
target_labels = le.inverse_transform(target2)
data2['Nivel_MHI'] = target_labels

In [28]:
result_df

,Atributo,Ganho de Informação,Ganho de Informação Normalizado
26,BP,0.055492,1.000000
1,C,0.039201,0.706079
23,AZ,0.036363,0.654880
7,O,0.022827,0.410669
13,AO,0.021020,0.378063
5,M,0.020295,0.364981
14,AP,0.017528,0.315055
44,J_5.0,0.016862,0.303043
19,AV,0.015995,0.287401
25,BO,0.015834,0.284501


In [29]:
# Calcular os ganhos de informação
result_df = calculate_information_gains(data2, 'Nivel_MHI')

#Copiando Coordenadas
result_df['Coordenadas'] = result_df['Atributo']

# Copiar "Atributo" para poder mapear para questões
result_df['Questionario'] = result_df['Coordenadas']

# Traduzir os índices para os nomes originais das colunas
result_df['Atributo'] = result_df['Atributo'].map(column_titles)
result_df

,Atributo,Ganho de Informação,Coordenadas,Questionario
81,DOMINIO 2(PSICOLOGICO),0.496086,CS,CS
80,DOMINIO 1(FISICO),0.338931,CR,CR
79,Com que frequência você tem sentimentos negati...,0.317214,CQ,CQ
72,Quão satisfeito(a) você está consigo mesmo?,0.309827,CJ,CJ
61,Quão seguro(a) você se sente em sua vida diária?,0.267769,BY,BY
...,...,...,...,...
2,Local de origem(Sobral),0.000224,F,F
51,Transporte para à Universidade(Transporte cole...,0.000197,AB_2.0,AB_2.0
18,Atividade de Lazer(Assitir filmes/ séries),0.000149,AU,AU
34,Situação Civil(União estável),0.000102,E_3.0,E_3.0


In [30]:
# Traduzir os índices para os nomes originais das colunas
result_df['Questionario'] = result_df['Questionario'].map(questionarios)
result_df

,Atributo,Ganho de Informação,Coordenadas,Questionario
81,DOMINIO 2(PSICOLOGICO),0.496086,CS,WHOQOL
80,DOMINIO 1(FISICO),0.338931,CR,WHOQOL
79,Com que frequência você tem sentimentos negati...,0.317214,CQ,WHOQOL
72,Quão satisfeito(a) você está consigo mesmo?,0.309827,CJ,WHOQOL
61,Quão seguro(a) você se sente em sua vida diária?,0.267769,BY,WHOQOL
...,...,...,...,...
2,Local de origem(Sobral),0.000224,F,Sociodemografico
51,Transporte para à Universidade(Transporte cole...,0.000197,AB_2.0,Sociodemografico
18,Atividade de Lazer(Assitir filmes/ séries),0.000149,AU,Sociodemografico
34,Situação Civil(União estável),0.000102,E_3.0,Sociodemografico


In [34]:
# Calcular ganho de informação da tabela e a soma dos ganhos de informação.
result_df['Ganho de Informação'] = result_df['Ganho de Informação'].astype(float)
result_df['Ganho de Informação Normalizado'] = result_df['Ganho de Informação Normalizado'].astype(float)
result_df['Ganho de Informação'] = result_df['Ganho de Informação'].astype(float)
result_df['Ganho de Informação Normalizado'] = result_df['Ganho de Informação Normalizado'].astype(float)
result_df['Ganho de Informação'] = result_df['Ganho de Informação'].astype(float)


In [35]:
result_df

,Atributo,Ganho de Informação,Coordenadas,Questionario,Ganho de Informação Normalizado
81,DOMINIO 2(PSICOLOGICO),0.496086,CS,WHOQOL,1.000000
80,DOMINIO 1(FISICO),0.338931,CR,WHOQOL,0.683168
79,Com que frequência você tem sentimentos negati...,0.317214,CQ,WHOQOL,0.639386
72,Quão satisfeito(a) você está consigo mesmo?,0.309827,CJ,WHOQOL,0.624493
61,Quão seguro(a) você se sente em sua vida diária?,0.267769,BY,WHOQOL,0.539702
...,...,...,...,...,...
2,Local de origem(Sobral),0.000224,F,Sociodemografico,0.000319
51,Transporte para à Universidade(Transporte cole...,0.000197,AB_2.0,Sociodemografico,0.000265
18,Atividade de Lazer(Assitir filmes/ séries),0.000149,AU,Sociodemografico,0.000167
34,Situação Civil(União estável),0.000102,E_3.0,Sociodemografico,0.000074
